In [ ]:
# BigQuery y Pandas
import pandas as pd
from google.cloud import bigquery
import json

# Web Scrapping
import os
import requests
from bs4 import BeautifulSoup

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'keys\nyc-taxis-co2-0900a053e59f.json'  # json file with credentials

In [ ]:
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
parquet_files = []

# selecting the years
selected_years = ['2020', '2021', '2022','2023']

for link in soup.find_all('a',href=True): # finding links
  if link['href'].endswith('.parquet '): # Si el elemento termina en .parquet, añado el elemento a la lista parquet_files
    if any(year in link['href'] for year in selected_years):
      print(link['href'])
      parquet_files.append(link['href'])

In [ ]:
yellow_taxis = []
green_taxis = []

for link in parquet_files:
  if 'yellow_tripdata' in link: # Todos los links que contengan 'yellow_tripdata' en su texto
    yellow_taxis.append(link)
  elif 'green_tripdata' in link: # Todos los links que contengan 'green_tripdata' en su texto
    green_taxis.append(link)


yellow_taxis = sorted(yellow_taxis, reverse=True)
green_taxis = sorted(green_taxis, reverse=True)


In [ ]:
yellow_taxis

In [ ]:
green_taxis

In [ ]:
# IdLocation de Manhattan
manhattan_zones = [  4,  12,  13,  24,  41,  42,  43,  45,  48,  50,  68,  74,  75,
    79,  87,  88,  90, 100, 103, 104, 105, 107, 113, 114, 116, 120,
    125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
    158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224,
    229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246,
    249, 261, 262, 263]

## TAXIS AMARILLOS

In [ ]:
dataframes = []
for parquet_file in yellow_taxis:
    df = pd.read_parquet(parquet_file)
    df = df[['tpep_pickup_datetime','PULocationID','DOLocationID']]

    # Selecciono solamente los registros que pertenecen a viajes del distrito de Manhattan
    df = df[df.PULocationID.isin(manhattan_zones) & df.DOLocationID.isin(manhattan_zones)].reset_index(drop=True)
    dataframes.append(df)

# Ahora concateno todos esos dataframes en uno solo
full_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
# Configura el cliente de BigQuery
client = bigquery.Client()

# Define el nombre de la tabla y el ID del proyecto
table_id = 'nyc-taxis-project.new_york_transport_project.yellow_taxis2'

# Carga el DataFrame en BigQuery
job_config = bigquery.LoadJobConfig()
job = client.load_table_from_dataframe(full_df, table_id, job_config=job_config)

# Espera a que se complete el job
job.result()

print(f"Se cargaron {job.output_rows} filas en la tabla {table_id}.")

## TAXIS VERDES

In [ ]:
dataframes = []
for parquet_file in green_taxis:
    df = pd.read_parquet(parquet_file)
    df = df[['lpep_pickup_datetime','PULocationID','DOLocationID']]

    # Selecciono solamente los registros que pertenecen a viajes del distrito de Manhattan
    df = df[df.PULocationID.isin(manhattan_zones) & df.DOLocationID.isin(manhattan_zones)].reset_index(drop=True)
    dataframes.append(df)

# Ahora concateno todos esos dataframes en uno solo
full_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
# Configura el cliente de BigQuery
client = bigquery.Client()

# Define el nombre de la tabla y el ID del proyecto
table_id = 'nyc-taxis-project.new_york_transport_project.green_taxis'

# Carga el DataFrame en BigQuery
job_config = bigquery.LoadJobConfig()
job = client.load_table_from_dataframe(full_df, table_id, job_config=job_config)

# Espera a que se complete el job
job.result()

print(f"Se cargaron {job.output_rows} filas en la tabla {table_id}.")